In [1]:
import tensorflow as tf

In [10]:
graph = tf.Graph()

with graph.as_default():
    #We've initialized a graph, but we still need to create a named 
    #Scope within the graph, lest things get messy.
    with tf.name_scope("variables"):
        
        #This is our counter variable. Counters should always have their 
        # 'trainable' value set to False so that optimizers will leave them
        # alone!
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, 
                                  name="global_step")
        
        #Variable that keeps track of the sum of all output 
        #values over time
        total_output=tf.Variable(0.0, dtype=tf.float32, trainable=False, 
                                name="total_output")
        
    with tf.name_scope("transformation"):
        
        #Separate input layer
        with tf.name_scope("input"):
            #Create input placeholder--takes in a Vector
            a = tf.placeholder(tf.float32, shape=[None], 
                               name="input_placeholder")
        #Separate middle layer
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name="product_b")
            c = tf.reduce_sum(a, name="sum_c")
        
        #Separate output layer
        with tf.name_scope("output"):
            output = tf.add(b, c, name="output")
            
       
    with tf.name_scope("update"):
        # Increments the total_output Variable by the latest output
        # (this variable just acts as an accumulator)
        update_total = total_output.assign_add(output)
        
        #Increments the above 'global step' Variable, should be 
        #run whenever the graph is run.
        increment_step = global_step.assign_add(1)
        
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32),
                    name="average")
        
        #Creates summaries for output node
        tf.summary.scalar('Output', output)
        tf.summary.scalar('Sum of outputs over time', update_total)
        tf.summary.scalar('Average of outputs over time', avg)
    
    with tf.name_scope('global_ops'):
        # Initialization Op
        init = tf.global_variables_initializer()
        # Merge all summaries into one Operation
        merged_summaries = tf.summary.merge_all()
    
sess  = tf.Session(graph=graph)
writer = tf.summary.FileWriter('./improved_graph', graph)

sess.run(init)        

INFO:tensorflow:Summary name Sum of outputs over time is illegal; using Sum_of_outputs_over_time instead.
INFO:tensorflow:Summary name Average of outputs over time is illegal; using Average_of_outputs_over_time instead.


In [11]:
# Create helper function for running graphs
def run_graph(input_tensor):
    feed_dict = {a: input_tensor}
    _, step, summary = sess.run([output, increment_step, 
                                 merged_summaries], 
                                feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)
    

In [12]:
run_graph([2, 8])
run_graph([3, 1, 3, 3])

In [13]:
# Let's write the summaries to disk by using the .flush() method
writer.flush()
writer.close()
sess.close()